In [1]:
import os
import glob
import pandas as pd
import numpy

# Function to read in Smart Sheet data frames

In [35]:
def project(choice, env_choice):
    """This will load in Central or nothern Data Frame and set it up for merging depending on what we QCing too"""
    
    # load in data frame and most recent download
    
    if choice == 'CENTRAL':
        path = r'C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*'
        file_type = '.xlsx'
        loadUp1 = glob.glob(path + file_type)
        df = pd.read_excel(max(loadUp1, key=os.path.getctime))
        
    elif choice == 'NORTHERN':
        path = r"C:\Users\jacque.trahan\Downloads\Northern Division 2021 Debris Removal Tracker*"
        file_type = '.xlsx'
        loadUp2 = glob.glob(path + file_type)
        df = pd.read_excel(max(loadUp2, key=os.path.getctime))
    
    else:
        print('invalid entry re-run')
        
        
      # set up data frame columns based on ev check we at doing
    
    if env_choice == 'ASB':
               # SMART SHEET ASB SET UP

        smartsheets = df.copy()

        # changes date column from object to a date.and fill na with 0's
        smartsheets["ASB Assessment"] = smartsheets["ASB Assessment"].dt.date.fillna(0)
        smartsheets["Chimney Tipped"] = smartsheets["Chimney Tipped"].dt.date.fillna(0)
        smartsheets["ASB Abatement"] = smartsheets["ASB Abatement"].dt.date.fillna(0)

        # rename columns for better comparing
        smartsheets.rename(columns={'ASB Assessment': 'Date Collected', 'Chimney Tipped':'CHIMNEY TIP DATE',
                                'County': 'SS_COUNTY'}, inplace=True)

        #make all column headers upper
        smartsheets.columns = map(str.upper, smartsheets.columns)

        # Column section normal uppercase
        ss_cols = smartsheets[['APN_ROW SEGMENT','STREET #','STREET NAME','STRUCTURAL STATUS','SS_COUNTY',
                           'ASB RESULTS',
                         'CHIMNEY','CHIMNEY TIP DATE',
                         'NESHAP WALLS', 
                         "DATE COLLECTED",'ASB SUSPECTED',
                         'ASB ABATEMENT']] 
        return ss_cols
    
    elif env_choice == 'SA': 
        
        smartsheets = df.copy()

        # take of the time on the dates
        smartsheets['Site Assessment'] = smartsheets['Site Assessment'].dt.date
        
        # fillna with 0s
        smartsheets.fillna({'Chimney': 'No',
                    'Site Assessment': 0,
                    'Number of Passenger Vehicles': 0,
                    'Number of ATV_UTV_snowmobiles': 0,
                    'Number of Ag_Commercial_Construction Equipment': 0,
                    'Number of Motorcycles': 0,
                    'Number of RV\'s':0,
                    'Number of Trailers': 0,
                    'Number of Vessels (Boat_Jetski)': 0}, inplace=True)
        
                                # smart sheet column selection
        smartsheets = smartsheets[['APN_ROW Segment','Street #','Street Name', 'Structural Status','County',
                           'Chimney',
                           'Site Assessment',
                          'Number of Passenger Vehicles', 'Number of ATV_UTV_snowmobiles',
                          'Number of Ag_Commercial_Construction Equipment','Number of Motorcycles',
                          'Number of RV\'s', 'Number of Trailers', 'Number of Vessels (Boat_Jetski)', 'Total Number of Vehicles']]
        
        #rename columns for easier readability
        smartsheets.rename(columns={'County':'County_SS','Site Assessment': 'SA Survey Date', 'Number of Passenger Vehicles': 'Automobiles',
                           'Number of ATV_UTV_snowmobiles': 'ATV/UTV/Snowmobiles',
                           'Number of Ag_Commercial_Construction Equipment': 'Ag/Commercial/Construction',
                           'Number of Motorcycles': 'Motorcycles_SS', 'Number of RV\'s': 'RV_SS',
                           'Number of Trailers':'Trailers_SS','Number of Vessels (Boat_Jetski)':'Vessel_SS'}, inplace=True)
        
        smartsheets.columns = map(str.upper, smartsheets.columns)
        
        ss_cols = smartsheets.copy()

        return ss_cols
    
    

In [44]:
def ASB(project_choice):
    """Run the asbestos audit with the selected data frame. Central or Northern"""
    
    #Gets Asbestos tracker
    path2 = r"C:\Users\jacque.trahan\Downloads\Asbestos Tracker*"
    file_type2 = ".csv"
    load_it2 = glob.glob(path2 + file_type2)
    df2 = pd.read_csv(max(load_it2, key=os.path.getctime))
    
        # ASBESTOS SETUP
    
    
    
    if project_choice == 'CENTRAL':
        asbestos = df2.copy()
        # Take out all divisions other than central.
        asbestos = asbestos[asbestos['Division'].isin(['Central Division'])]

        # changes date column from object to a date.
        asbestos['Date Collected'] = pd.to_datetime(asbestos['Date Collected']).dt.date
        asbestos['Chimney Tip Date'] = pd.to_datetime(asbestos['Chimney Tip Date']).dt.date
        asbestos['Chimney Abatement Completed Date'] = pd.to_datetime(asbestos['Chimney Abatement Completed Date']).dt.date
        asbestos['Asbestos Abatement Completed Date'] = pd.to_datetime(asbestos['Asbestos Abatement Completed Date']).dt.date
        asbestos['Date Shipped or Driven'] = pd.to_datetime(asbestos['Date Shipped or Driven']).dt.date

         #make all column headers upper
        asbestos.columns = map(str.upper, asbestos.columns)

         # rename columns for better comparing
        asbestos.rename(columns={'APN':'APN_ASB'}, inplace=True)

        # Column section all upper case
        asb_cols = asbestos[['APN_ASB', 'FINAL RESULTS', 'CHIMNEY', 'CHIMNEY TIP DATE', 'CHIMNEY FINDING',
                     'NESHAP WALLS',
                     'DATE COLLECTED', 'DATE SHIPPED OR DRIVEN', 
                     'SAMPLE QUANTITY', 'POINT COUNT NEEDED', 'POINT COUNT RESULTS',
                     'ASBESTOS ABATEMENT COMPLETED DATE', 'CHIMNEY ABATEMENT COMPLETED DATE',
                     'PLANNING NOTES', 'DIVISION']]




        # gets the ESLs grabs all files in this folder and concats them
        path = r"C:\Users\jacque.trahan\Desktop\Dixie Fires Audit\ESLs\Central"
        esl_final1 = pd.concat(map(pd.read_excel, glob.glob(path + '/*.xlsx')))

        esl_final = esl_final1.copy()
        #esl setup, change names,formate dates, and fillna's with 0. End with index reset
        esl_final = esl_final[['APN', 'DATE_SHIPPED_OR_DRIVEN']]
        esl_final.rename(columns={'DATE_SHIPPED_OR_DRIVEN':'DATE_SHIPPED_OR_DRIVEN_esl'}, inplace=True)
        esl_final['DATE_SHIPPED_OR_DRIVEN_esl'] = pd.to_datetime(esl_final['DATE_SHIPPED_OR_DRIVEN_esl']).dt.date.fillna(0)
        esl = esl_final.groupby(['APN'])[['DATE_SHIPPED_OR_DRIVEN_esl']].max().reset_index()



        # merge test esl asb and ss
        merge1 = asb_cols.merge(esl,
                              left_on='APN_ASB',
                              right_on='APN',
                              how='outer',
                              suffixes=('_SS','_ASB'))

        ASB_ESLs = merge1.copy()
    
    return ASB_ESLs

# Start Program 

In [45]:
project_choice = input('Central or Northern Dixie Fire: \n').upper()
env_choice = input('What are we QCing? "ASB", "SA", "ARCH", "BIO": \n ').upper()
smartsheets = project(project_choice, env_choice)

Central or Northern Dixie Fire: 
central
What are we QCing? "ASB", "SA", "ARCH", "BIO": 
 asb


In [46]:
if env_choice == "ASB":
    ASB_ESLs = ASB(project_choice)

In [47]:
ASB_ESLs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818 entries, 0 to 817
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   APN_ASB                            818 non-null    object        
 1   FINAL RESULTS                      397 non-null    object        
 2   CHIMNEY                            818 non-null    object        
 3   CHIMNEY TIP DATE                   29 non-null     object        
 4   CHIMNEY FINDING                    1 non-null      object        
 5   NESHAP WALLS                       818 non-null    object        
 6   DATE COLLECTED                     488 non-null    object        
 7   DATE SHIPPED OR DRIVEN             381 non-null    object        
 8   SAMPLE QUANTITY                    486 non-null    float64       
 9   POINT COUNT NEEDED                 818 non-null    object        
 10  POINT COUNT RESULTS                23 